In [2]:
import pandas as pd

import datetime

In [92]:
df_1 = pd.DataFrame({"a": [1, 2, 3, 4, 5]})
df_2 = pd.DataFrame({"b": [1, 2, 3, 4, 5]})
df_3 = pd.DataFrame({"c": [1, 2, 3, 4, 5]})

In [94]:
pd.concat([df_1, df_2.shift(1), df_3.shift(2)], axis=1)

,a,d,b,c
0,1,1,NaN,NaN
1,2,2,1.0,NaN
2,3,3,2.0,1.0
3,4,4,3.0,2.0
4,5,5,4.0,3.0


In [56]:
dfm = pd.read_csv("datasets/EURUSD/15m.csv", nrows=2000, date_format=lambda x: pd.to_datetime(x, dayfirst=True)).drop(["Volume"], axis=1)
dfh = pd.read_csv("datasets/EURUSD/1h.csv", nrows=1000, date_format=lambda x: pd.to_datetime(x, dayfirst=True)).drop(["Volume"], axis=1)

In [41]:
time_column = "Date"
window_size = 2

In [15]:
df = pd.DataFrame()

In [57]:
valid = dfm["Date"].isin(dfh["Date"])
stacked = pd.concat([dfm.add_suffix(f"_{i}").shift(i) for i in range(window_size)], axis=1)

stacked = stacked[valid].reset_index(drop=True)
#pd.concat([dfh, stacked], axis=1).head()

In [87]:
def processor(row: pd.Series, window_size: int) -> pd.Series:
    for i in range(window_size):
        row[f"Change_{i}"] = ((row[f"Close_{i}"] - row[f"Open_{i}"]) / row[f"Open_{i}"]) * 100
    return row[[f"Change_{i}" for i in range(window_size)]]

In [88]:
stacked.apply(lambda row: processor(row, window_size), axis=1).head()

,Change_0,Change_1
0,-0.014979,NaN
1,0.000936,0.011232
2,0.030880,0.008423
3,-0.005616,-0.023396
4,-0.004682,-0.002809
